# RLC circuit parameters visualization

Interactive 3D visualization of an RLC circuit function parameters by using [ipywidgets](https://pypi.org/project/ipywidgets/).

## Oscillating RLC scenario

The RLC function is a sum of two conjugate Euler's functions:
$$e^{\Omega t + \Phi} + e^{\overline\Omega t + \overline\Phi} = e^{\Omega t + \Phi} + \overline{e^{\Omega t + \Phi}}$$

Which is also the real component of a single Euler's function:
$$\Re{e^{\Omega t + \Phi}} = \frac{e^{\Omega t + \Phi} + \overline{e^{\Omega t + \Phi}}}{2}$$

## Non-oscillating RLC scenario

The RLC function is a sum of two regular exponent functions:
$$e^{a t + a_0} + e^{b t + b_0}$$

Where $a$, $b$ and $a_0$, $b_0$ can also be expressed similarly to the conjugates:
$$\begin{cases}a = m+d & a_0 = m_0+d_0 \\ b = m-d & b_0 = m_0-d_0 \end{cases}$$


In [ ]:
import numpy as np
import ipywidgets
import plotly.graph_objects as go
import rlc_funcs
import rlc_reverse

NUM_T_STEPS = 40
NUM_SAMPLES = 4
PM = np.array((1, -1))

fig = go.FigureWidget(layout=dict(
            # Suitable for 3D scatters
            height=600,
            margin={'b': 4, 'l': 4, 'r': 4, 't': 4},
            legend_x=0,
            scene = dict(
                    aspectratio=dict(x=1, y=2, z=1),
                    xaxis_title='imag (x)',
                    yaxis_title='time (y)',
                    zaxis_title='real (z)'),
            title_x=.5,
        ))
# Emphasis the axes
fig.update_layout(scene=dict(
        xaxis_zerolinecolor='gray',
        yaxis_zerolinecolor='gray',
        zaxis_zerolinecolor='gray',
    ))

# Summary annotation
fig.add_annotation(x=1, y=.9, xref='paper', yref='paper', showarrow=False)
# Warning annotation
fig.add_annotation(x=.1, y=.1, xref='paper', yref='paper', showarrow=False, visible=False, bgcolor='red')
info_annot = fig.layout.annotations[0]
warning_annot = fig.layout.annotations[1]

# Main functions
fn_scatters = []
fig.add_scatter3d(mode='lines', line_dash='dash', name=f'Euler\'s func')
fn_scatters.append(fig.data[-1])
fig.add_scatter3d(mode='lines', line_dash='dash', name=f'Euler\'s conj. func')
fn_scatters.append(fig.data[-1])

# Result (summed) function
fig.add_scatter3d(mode='lines', name=f'Result func')
fn_r_scatter = fig.data[-1]

# Sample values
fig.add_scatter3d(mode='markers', name=f'Sample data')
sample_scatter = fig.data[-1]
# At point0
sample0_scatters = []
fig.add_scatter3d(mode='lines', line_dash='dot', name=f'Sample<sub>0</sub>')
sample0_scatters.append(fig.data[-1])
fig.add_scatter3d(mode='lines', line_dash='dot', name=f'Sample<sub>0</sub> conj.')
sample0_scatters.append(fig.data[-1])
fig.add_scatter3d(mode='lines', line_dash='dash', name=f'Sample<sub>0</sub> result')
sample0_r_scatter = fig.data[-1]

# Reversed function from sample values
fig.add_scatter3d(mode='lines', line_dash='dashdot', line_color='red', name=f'Reversed func')
reversed_scatter = fig.data[-1]

def get_sample_vals(omegas, phis, trange, conjugated=False):
    """Reference Euler's function values"""
    vals = rlc_funcs.calc_euler_derivs(1, omegas, phis, trange)
    vals = vals[0]  # Take the only 0-th derivative
    if conjugated:
        vals_r = vals.mean(0)
    else:
        vals_r = vals.real
        vals = vals[np.newaxis, ...]
    return vals, vals_r

@ipywidgets.interact(
        a=(-10., 10.), b=(-2*np.pi, 4*np.pi),
        a0=(-5., 5.), b0=(-np.pi, np.pi),
        imag_b=True,
        add_conj=False,
        sample_dt=(1e-2, 2),
)
def update(a=-.5, b=np.pi/2, a0=0, b0=0, imag_b=True, add_conj=False, sample_dt=1/4):
    """Interactively update plot"""
    # Select trange [-<sample-period>, 3 <sample-periods>]
    trange = np.linspace(-NUM_SAMPLES*sample_dt, 3*NUM_SAMPLES*sample_dt, NUM_T_STEPS, endpoint=True)

    if imag_b:
        b = complex(0, b)
        b0 = complex(0, b0)
    if add_conj:
        b = b * PM
        b0 = b0 * PM
    omegas = np.asarray(a + b)
    phis = np.asarray(a0 + b0)

    info_annot.text = f'&#937; {np.round(omegas, 3)}, e<sup>&#937;</sup> {np.round(np.exp(omegas), 3)}'
    info_annot.text += f'<br>&#934; {np.round(phis, 3)}, e<sup>&#934;</sup> {np.round(np.exp(phis), 3)}<br>'
    if imag_b:
        info_annot.text += f'<br>Oscillation: {np.round(omegas.imag*180/np.pi,1)} deg/sec, {np.round(omegas.imag/2/np.pi, 2)} Hz' \
                f', {np.round(2*np.pi / (sample_dt * omegas.imag), 1)} samples/period'
    if (omegas.real != 0).any():
        if  (omegas.real <= 0).all():
            info_annot.text += f'<br>Attenuation: {np.round(1 / -omegas.real, 2)} sec'
        else:
            info_annot.text += f'<br>Intensification: {np.round(1 / omegas.real, 2)} sec'

    # Function values
    fn, fn_r = get_sample_vals(omegas, phis, trange, add_conj)

    # Regular/conjugated Euler's functions along z,x, time along y
    for i, scatt in enumerate(fn_scatters):
        scatt.y = trange
        if i < fn.shape[0]:
            scatt.z = fn[i].real
            scatt.x = fn[i].imag
        else:
            scatt.z = scatt.x = None

    # Result function along z,x, time along y
    fn_r_scatter.y = trange
    fn_r_scatter.z = fn_r.real
    fn_r_scatter.x = fn_r.imag

    # Sample data values
    strange = np.arange(NUM_SAMPLES) * sample_dt
    sfn, sfn_r = get_sample_vals(omegas, phis, strange, add_conj)
    sfn_r = sfn_r.real      # Ensure real component only
    sample_scatter.y = strange
    sample_scatter.z = sfn_r.real
    sample_scatter.x = sfn_r.imag
    sample_scatter.text = [f'Sample<sub>{i}</sub>' for i, _ in enumerate(sfn_r)]
    # Sample0 values
    for i, scatt in enumerate(sample0_scatters):
        scatt.y = [0,0]
        if i < sfn.shape[0]:
            scatt.y = [strange[0], strange[0]]
            scatt.z = [0, sfn[i][0].real]
            scatt.x = [0, sfn[i][0].imag]
        else:
            scatt.z = scatt.x = None
    sample0_r_scatter.y = [strange[0], strange[0]]
    sample0_r_scatter.z = [0, sfn_r[0].real]
    sample0_r_scatter.x = [0, sfn_r[0].imag]

    # Reverse the Euler's coeficients
    rev_omega, rev_phi = rlc_reverse.from_4samples(sfn_r, sample_dt, conjugated=add_conj)
    rev_fn, _  = get_sample_vals(rev_omega, rev_phi, trange, add_conj)
    rev_fn = rev_fn.mean(0)
    reversed_scatter.y = trange
    reversed_scatter.z = rev_fn.real
    reversed_scatter.x = rev_fn.imag
    info_annot.text += f'<br><br>Reversed: &#937; {np.round(rev_omega, 3)}, &#934; {np.round(rev_phi, 3)}'

    # Warning when reversed sample values don't match
    warning_annot.text = ''
    _, rev_sfn_r  = get_sample_vals(rev_omega, rev_phi, strange, add_conj)
    if np.round(rev_sfn_r - sfn_r, 8).any():
        idx = np.nonzero(np.round(rev_sfn_r - sfn_r, 8))[0]
        warning_annot.text += f'<br>Sample<sub>{idx}</sub> deviation: {np.round(rev_sfn_r[idx] - sfn_r[idx], 3)}'

    # Warning when reversed coefficients don't match
    if np.round(rev_omega - omegas, 8).any():
        warning_annot.text += f'<br>Omega deviation: {np.round(rev_omega, 3)}, actual {np.round(omegas, 3)}'
    if np.round(rev_phi - phis, 8).any():
        warning_annot.text += f'<br>Phase deviation: {np.round(rev_phi, 3)}, actual {np.round(phis, 3)}'
    warning_annot.visible = len(warning_annot.text) > 0

fig